In [ ]:

!pip install --upgrade --quiet google-generativeai SpeechRecognition pydub ffmpeg-python faiss-cpu gTTS fuzzywuzzy python-Levenshtein

import google.generativeai as genai
from google.colab import output, userdata
from base64 import b64decode
from pydub import AudioSegment
import speech_recognition as sr
import IPython.display as display
import pandas as pd
import faiss
from gtts import gTTS
import IPython.display as ipd
import numpy as np
from fuzzywuzzy import process


GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)


generative_model = genai.GenerativeModel(model_name="gemini-1.0-pro")


record_js = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  var reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})

var record = async function(){
  const stream = await navigator.mediaDevices.getUserMedia({audio:true})
  const recorder = new MediaRecorder(stream)
  var data = []

  recorder.ondataavailable = e => data.push(e.data)
  recorder.start()

  await sleep(5000) // 5 seconds
  recorder.stop()

  await new Promise(resolve => recorder.onstop = resolve)
  let blob = new Blob(data, {type:'audio/wav'})
  let text = await b2text(blob)

  return text
}
record()
"""

excel_file_path = '/content/Dataset for thesis.xlsx'
try:
    df = pd.read_excel(excel_file_path)
    print("🧾 Column names found in the dataset:")
    print(df.columns.tolist())

    df.dropna(subset=['রোগের নাম (বাংলা)', 'সাধারণ লক্ষণ', 'প্রাথমিক চিকিৎসা ও ঘরোয়া যত্ন'], inplace=True)
    df.reset_index(drop=True, inplace=True)
    df['combined_text'] = df['রোগের নাম (বাংলা)'].fillna('') + ' ' + df['সাধারণ লক্ষণ'].fillna('')
    print(f"✅ Dataset loaded. Found {len(df)} valid medical entries.")

    print("🧠 Creating embeddings for combined disease names and symptoms...")
    combined_embeddings = genai.embed_content(
        model='models/embedding-001',
        content=df['combined_text'].tolist(),
        task_type="RETRIEVAL_DOCUMENT"
    )['embedding']
    combined_embeddings = np.array(combined_embeddings).astype('float32')

    dimension = combined_embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(combined_embeddings)
    print("✅ FAISS index created successfully.")

except FileNotFoundError:
    print(f"❌ Error: The file was not found at '{excel_file_path}'. Please upload the file and check the path.")
    df = None
except Exception as e:
    print(f"❌ FATAL ERROR: An error occurred during data preparation or FAISS indexing: {e}")
    df = None


def retrieve_context(query_text, top_k=1, disease_match_threshold=80):
    """Searches the knowledge base with enhanced matching."""
    if df is None:
        print("❌ Cannot retrieve context: DataFrame is not loaded.")
        return None

    print(f"🔍 Searching for matches for: '{query_text}'")


    disease_names = df['রোগের নাম (বাংলা)'].tolist()
    best_match_disease, score = process.extractOne(query_text, disease_names)

    if score >= disease_match_threshold:
        print(f"✅ Fuzzy disease name match found: '{best_match_disease}' with score {score}")
        match_index = df[df['রোগের নাম (বাংলা)'] == best_match_disease].index[0]
        retrieved_info = df.loc[match_index]
        return {
            "disease_name": retrieved_info['রোগের নাম (বাংলা)'],
            "symptoms": retrieved_info['সাধারণ লক্ষণ'],
            "remedy": retrieved_info['প্রাথমিক চিকিৎসা ও ঘরোয়া যত্ন'],
            "match_type": "fuzzy_disease"
        }


    print("🔍 No strong fuzzy disease match, searching using FAISS on combined text...")

    try:

        query_embedding = genai.embed_content(
            model='models/embedding-001',
            content=query_text,
            task_type="RETRIEVAL_QUERY"
        )['embedding']
        query_embedding = np.array([query_embedding]).astype('float32')


        D, I = index.search(query_embedding, top_k)


        best_match_idx = I[0][0]
        best_distance = D[0][0]


        print(f"✅ FAISS search found best match at index {best_match_idx} with distance {best_distance}")


        if best_match_idx < len(df):
            retrieved_info = df.loc[best_match_idx]
            return {
                "disease_name": retrieved_info['রোগের নাম (বাংলা)'],
                "symptoms": retrieved_info['সাধারণ লক্ষণ'],
                "remedy": retrieved_info['প্রাথমিক চিকিৎসা ও ঘরোয়া যত্ন'],
                "match_type": "faiss_combined"
            }
        else:
            print("⚠️ FAISS search returned an invalid index.")
            return None

    except Exception as e:
        print(f"❌ Error during FAISS search: {e}")
        return None

def generate_response(user_query, context):
    """Generates a more precise response based on match type."""
    if context is None:
        return "দুঃখিত, আমি আপনার লক্ষণগুলির সাথে মিলে যায় এমন কোনো রোগ খুঁজে পাইনি। অনুগ্রহ করে একজন ডাক্তারের সাথে পরামর্শ করুন।"

    print("🤖 Generating response...")

    if context['match_type'] == "fuzzy_disease":
        prompt = f"""
        আপনি একজন বন্ধুত্বপূর্ণ চিকিৎসা সহকারী। ব্যবহারকারী সম্ভবত এই রোগ সম্পর্কে জানতে চায়: "{context['disease_name']}"।

        রোগের নাম: {context['disease_name']}
        সাধারণ লক্ষণ: {context['symptoms']}
        প্রাথমিক চিকিৎসা ও ঘরোয়া যত্ন: {context['remedy']}

        অনুগ্রহ করে নিম্নলিখিত তথ্য প্রদান করুন:
        1. রোগটির সংক্ষিপ্ত বর্ণনা
        2. প্রধান লক্ষণগুলো
        3. ঘরোয়া প্রতিকার/প্রাথমিক চিকিৎসা
        4. সতর্কতা: এটি শুধুমাত্র প্রাথমিক পরামর্শ, ডাক্তারের পরামর্শ প্রয়োজন
        """
    else:
        prompt = f"""
        ব্যবহারকারী নিম্নলিখিত লক্ষণগুলি বর্ণনা করেছেন: "{user_query}"

        আমাদের ডাটাবেস অনুসারে, এটি সম্ভবত {context['disease_name']} রোগের সাথে সম্পর্কিত।

        রোগের নাম: {context['disease_name']}
        সাধারণ লক্ষণ: {context['symptoms']}
        প্রাথমিক চিকিৎসা ও ঘরোয়া যত্ন: {context['remedy']}

        অনুগ্রহ করে নিম্নলিখিত তথ্য প্রদান করুন:
        1. ব্যবহারকারীর বর্ণিত লক্ষণগুলির স্বীকৃতি
        2. সম্ভাব্য রোগের নাম ও তার লক্ষণ
        3. প্রাথমিক চিকিৎসা ও ঘরোয়া প্রতিকার
        4. জোর দিয়ে বলুন যে এটি শুধুমাত্র প্রাথমিক পরামর্শ এবং ডাক্তারের সাথে পরামর্শ আবশ্যক
        """

    try:
        response = generative_model.generate_content(prompt)
        return response.text
    except Exception as e:
        print(f"❌ Error during response generation: {e}")

        return f": {context['remedy']}"



if df is not None and 'index' in locals():

    print("\n🎤 অনুগ্রহ করে আপনার রোগের নাম বলুন অথবা আপনার লক্ষণগুলো বর্ণনা করুন (৫ সেকেন্ড রেকর্ডিং)...")
    audio_data = output.eval_js(record_js)
    audio_bytes = b64decode(audio_data.split(',')[1])
    with open("bangla_speech.wav", "wb") as f:
        f.write(audio_bytes)

    display.display(display.Audio("bangla_speech.wav", autoplay=True))

    audio = AudioSegment.from_file("bangla_speech.wav")
    audio = audio.set_frame_rate(16000).set_channels(1)
    audio.export("bangla_fixed.wav", format="wav")

    recognizer = sr.Recognizer()
    user_query_text = ""
    try:
        with sr.AudioFile("bangla_fixed.wav") as source:
            audio_data_sr = recognizer.record(source)
        user_query_text = recognizer.recognize_google(audio_data_sr, language="bn-BD")
        print(f"🗣️  আপনি বলেছেন: \"{user_query_text}\"")
    except sr.UnknownValueError:
        print("⚠️ দুঃখিত, আমি আপনার কথা বুঝতে পারিনি। অনুগ্রহ করে আবার চেষ্টা করুন।")
    except sr.RequestError as e:
        print(f"❌ Google STT error: {e}")

    if user_query_text:

        retrieved_context = retrieve_context(user_query_text)

        if retrieved_context:
            print(f"✅ পাওয়া গেছে: '{retrieved_context['disease_name']}' ({retrieved_context['match_type']})")


            final_response = generate_response(user_query_text, retrieved_context)
            print("\n\n---\n📝 চূড়ান্ত পরামর্শ:\n---")
            print(final_response)


            print("\n\n🔊 পরামর্শটি শোনাচ্ছি...")
            try:
                tts = gTTS(text=final_response, lang='bn')
                tts.save("response.mp3")
                ipd.display(ipd.Audio("response.mp3", autoplay=True))
            except Exception as e:
                print(f"❌ Error during Text-to-Speech generation: {e}")
        else:
            print("দুঃখিত, আমি আপনার লক্ষণগুলির সাথে মিলে যায় এমন কোনো রোগ খুঁজে পাইনি। অনুগ্রহ করে একজন ডাক্তারের সাথে পরামর্শ করুন।")

🧾 Column names found in the dataset:
['ক্রমিক নং', 'রোগের নাম (বাংলা)', 'রোগের নাম (ইংরেজি)', 'সংজ্ঞা ও সংক্ষিপ্ত বিবরণ', 'সাধারণ লক্ষণ', 'কারণ', 'প্রতিরোধ', 'প্রাথমিক চিকিৎসা ও ঘরোয়া যত্ন', 'কখন চিকিৎসকের পরামর্শ জরুরি (Red Flags)', 'রোগের ধরণ/বিভাগ', 'তথ্যসূত্র']
✅ Dataset loaded. Found 68 valid medical entries.
🧠 Creating embeddings for combined disease names and symptoms...
✅ FAISS index created successfully.

🎤 অনুগ্রহ করে আপনার রোগের নাম বলুন অথবা আপনার লক্ষণগুলো বর্ণনা করুন (৫ সেকেন্ড রেকর্ডিং)...


🗣️  আপনি বলেছেন: "ডায়রিয়া"
🔍 Searching for matches for: 'ডায়রিয়া'
✅ Fuzzy disease name match found: 'ডায়রিয়া' with score 100
✅ পাওয়া গেছে: 'ডায়রিয়া' (fuzzy_disease)
🤖 Generating response...


❌ Error during response generation: 404 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.0-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: models/gemini-1.0-pro is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods.


---
📝 চূড়ান্ত পরামর্শ:
---
দুঃখিত, পরামর্শ তৈরি করতে সমস্যা হচ্ছে। প্রাপ্ত তথ্য ছিল: প্রচুর পরিমাণে তরল পান করা, যেমন- খাবার স্যালাইন (ORS), ডাবের পানি। সহজে হজমযোগ্য খাবার (যেমন- কলা, ভাত, জাউ) খাওয়া। মসলাযুক্ত ও চর্বিযুক্ত খাবার এড়িয়ে চলা।


🔊 পরামর্শটি শোনাচ্ছি...
